In [1]:
import pandas as pd
import warnings

warnings.filterwarnings("ignore", message="The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.", category=FutureWarning)

In [2]:
DATA_PATH = 'D:\\0Code\\UNSW\\2024Kaggle\\03_NCA_Comp\\dataset\\'

In [3]:
RSresultW = DATA_PATH+'06_WRegularSeasonDetailedResults.csv'
NCAresultW = DATA_PATH+'07_WNCAATourneyDetailedResults.csv'
RSresultW_df = pd.read_csv(RSresultW)
NCAresultW_df = pd.read_csv(NCAresultW)

In [4]:
RSresultW_df['ScoreDiff'] = RSresultW_df['WScore'] - RSresultW_df['LScore']
RSresultW_df.head(5)

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,ScoreDiff
0,2010,11,3103,63,3237,49,H,0,23,54,...,6,10,11,27,11,23,7,6,19,14
1,2010,11,3104,73,3399,68,N,0,26,62,...,14,27,14,26,7,20,4,2,27,5
2,2010,11,3110,71,3224,59,A,0,29,62,...,19,23,17,23,8,15,6,0,15,12
3,2010,11,3111,63,3267,58,A,0,27,52,...,16,25,22,22,15,11,14,5,14,5
4,2010,11,3119,74,3447,70,H,1,30,74,...,11,21,21,32,12,14,4,2,14,4


In [5]:
NCAresultW_df['ScoreDiff'] = NCAresultW_df['WScore'] - NCAresultW_df['LScore']
NCAresultW_df.head(5)

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,ScoreDiff
0,2010,138,3124,69,3201,55,N,0,28,57,...,3,5,17,19,12,18,4,1,18,14
1,2010,138,3173,67,3395,66,N,0,23,59,...,14,15,18,26,8,8,8,6,22,1
2,2010,138,3181,72,3214,37,H,0,26,57,...,3,8,10,21,4,16,6,4,20,35
3,2010,138,3199,75,3256,61,H,0,25,63,...,17,22,16,21,13,16,5,4,24,14
4,2010,138,3207,62,3265,42,N,0,24,68,...,11,17,16,22,9,10,3,4,12,20


In [6]:
Wresult = pd.concat([RSresultW_df, NCAresultW_df]).reset_index(drop=True)

In [7]:
all_teams = pd.unique(pd.concat([Wresult['WTeamID'], Wresult['LTeamID']]))
print(f"The total number of teams is {len(all_teams)}.")

The total number of teams is 364.


In [8]:
data = Wresult.copy()

years = data['Season'].unique()

new_data = pd.DataFrame(columns=['TeamID', 'Season', 'Score', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'ScoreVar', 
                                 'OppScore', 'OppFGM', 'OppFGA', 'OppFGM3', 'OppFGA3', 'OppFTM', 'OppFTA', 'OppStl', 'OppBlk', 'WinPct'])

for year in years:
    # 获取当前年份的数据
    year_data = data[data['Season'] == year]
    
    # 获取当前年份参与比赛的球队ID
    all_teams = pd.unique(pd.concat([year_data['WTeamID'], year_data['LTeamID']]))
    
    # 对每个球队进行处理
    for team_id in all_teams:
        # 处理胜利数据
        win_data = year_data[year_data['WTeamID'] == team_id]
        win_stats = win_data[['WScore', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk']]
        win_stats.columns = ['Score', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk']
        
        # 处理失败数据
        loss_data = year_data[year_data['LTeamID'] == team_id]
        loss_stats = loss_data[['LScore', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LStl', 'LBlk']]
        loss_stats.columns = ['Score', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'Stl', 'Blk']
        
        # 获取对手数据
        win_opponent_stats = win_data[['LScore', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LStl', 'LBlk']]
        win_opponent_stats.columns = ['OppScore','OppFGM', 'OppFGA', 'OppFGM3', 'OppFGA3', 'OppFTM', 'OppFTA', 'OppStl', 'OppBlk']
        loss_opponent_stats = loss_data[['WScore', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WStl', 'WBlk']]
        loss_opponent_stats.columns = ['OppScore', 'OppFGM', 'OppFGA', 'OppFGM3', 'OppFGA3', 'OppFTM', 'OppFTA', 'OppStl', 'OppBlk']
        
        # 合并胜利和失败数据
        team_stats = pd.concat([win_stats, loss_stats], axis=0)
        opponent_stats = pd.concat([win_opponent_stats, loss_opponent_stats], axis=0)
        
        # 计算平均数
        avg_stats = team_stats.mean(axis=0)
        
        # 计算方差
        var_score = team_stats['Score'].var()
        
        # 计算胜率
        total_games = len(win_data) + len(loss_data)
        win_pct = len(win_data) / total_games if total_games > 0 else 0
        
        # 添加TeamID列、Season列、方差列和胜率列
        avg_stats['TeamID'] = team_id
        avg_stats['Season'] = year
        avg_stats['ScoreVar'] = var_score
        avg_stats['WinPct'] = win_pct
        
        # 添加对手统计数据列
        avg_stats = pd.concat([avg_stats, opponent_stats.mean(axis=0)], axis=0)
        
        # 将计算结果添加到新的数据框中
        new_data = new_data.append(avg_stats, ignore_index=True)

# 重新排列列的顺序
new_data = new_data[['TeamID', 'Season', 'Score', 'WinPct', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'ScoreVar', 
                     'OppScore', 'OppFGM', 'OppFGA', 'OppFGM3', 'OppFGA3', 'OppFTM', 'OppFTA', 'OppStl', 'OppBlk']]

# 将 TeamID 列的数据类型转换为与原始数据一致的类型
new_data['TeamID'] = new_data['TeamID'].astype(data['WTeamID'].dtype)
new_data['Season'] = new_data['Season'].astype(data['Season'].dtype)



In [9]:
new_data

,TeamID,Season,Score,WinPct,FGM,FGA,FGM3,FGA3,FTM,FTA,...,ScoreVar,OppScore,OppFGM,OppFGA,OppFGM3,OppFGA3,OppFTM,OppFTA,OppStl,OppBlk
0,3103,2010,62.233333,0.566667,22.233333,55.266667,3.933333,12.466667,13.833333,19.600000,...,87.219540,59.633333,20.966667,54.966667,4.833333,15.166667,12.866667,18.833333,9.233333,3.800000
1,3104,2010,64.137931,0.379310,24.724138,62.103448,4.344828,14.724138,10.344828,16.793103,...,171.408867,67.000000,23.896552,60.137931,5.103448,16.517241,14.103448,21.965517,8.758621,3.586207
2,3110,2010,60.428571,0.619048,21.571429,53.904762,4.666667,15.285714,12.619048,16.809524,...,132.657143,54.380952,20.000000,52.904762,4.095238,14.190476,10.285714,15.000000,5.714286,2.666667
3,3111,2010,66.419355,0.612903,25.774194,59.483871,4.290323,12.387097,10.580645,16.612903,...,92.651613,63.838710,21.516129,59.193548,6.129032,18.645161,14.677419,20.548387,9.741935,3.516129
4,3119,2010,52.291667,0.333333,19.166667,53.541667,4.625000,16.916667,9.333333,13.875000,...,90.737319,58.750000,20.750000,54.750000,4.541667,13.166667,12.708333,18.458333,7.875000,3.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5235,3141,2024,60.583333,0.208333,22.333333,58.416667,5.291667,19.000000,10.625000,16.333333,...,59.297101,72.125000,26.000000,64.041667,7.458333,21.541667,12.666667,17.291667,10.291667,2.958333
5236,3476,2024,51.846154,0.153846,18.692308,55.576923,5.423077,19.923077,9.038462,12.961538,...,110.055385,68.230769,26.538462,61.500000,5.153846,17.500000,10.000000,14.846154,11.115385,3.423077
5237,3214,2024,55.041667,0.083333,19.416667,56.500000,3.000000,12.375000,13.208333,20.416667,...,84.389493,69.166667,23.083333,56.541667,6.541667,20.583333,16.458333,22.500000,8.541667,3.416667
5238,3152,2024,57.192308,0.000000,21.538462,59.961538,2.461538,10.769231,11.653846,16.153846,...,211.681538,84.769231,31.038462,67.846154,6.730769,21.153846,15.961538,23.538462,11.615385,2.423077


In [10]:
new_data.to_csv('001_Wdata.csv', index=False)